<a href="https://colab.research.google.com/github/pscspy/public/blob/main/arquivado/001/001_Web_Scraping_(FundsExplorer).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Web Scraping FII
O Script realiza um Web Scraping do site FundsExplorer, selecionando informações específicas de ativos de interesse e as contatenando em uma lista. Para o exemplo, considerou-se ativos aleatórios presentes no iFIX e informações relevantes:
* Ultimos dividendos (ultimo mês, três meses, seis e doze);
* Patrimônio Líquido
* Valor patrimonial por cota

In [1]:
#@title 1. Import
#@markdown Importando e checando (200) reposta do site.
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = requests.get('https://www.fundsexplorer.com.br/ranking')
url

<Response [200]>

In [2]:
#@title 2. Criando DataFrame
#@markdown Table to pd.DataFrame e exibindo lista. 
#   Investigating.
url_bs = BeautifulSoup(url.text,'html.parser')
#   Selecting table.
url_table = url_bs.findAll(attrs={'id':'scroll-wrapper'})
#   Table to Bs (Indexing) - type(table)
table = url_table[0].findAll('table')
#   Table to DataFrame
fii_df = pd.read_html(str(table))[0]
#   Lista por ordem alfabética
str(list(fii_df['Códigodo fundo'].sort_values()))

"['ABCP11', 'AFHI11', 'AFOF11', 'AGRX11', 'AIEC11', 'ALMI11', 'ALZR11', 'APTO11', 'ARCT11', 'ARRI11', 'ATSA11', 'BARI11', 'BBFI11B', 'BBFO11', 'BBGO11', 'BBIM11', 'BBPO11', 'BBRC11', 'BCFF11', 'BCIA11', 'BCRI11', 'BICE11', 'BICR11', 'BIME11', 'BLCA11', 'BLCP11', 'BLMC11', 'BLMG11', 'BLMO11', 'BLMR11', 'BMLC11', 'BNFS11', 'BPFF11', 'BPML11', 'BRCO11', 'BRCR11', 'BREV11', 'BRLA11', 'BTAL11', 'BTCR11', 'BTLG11', 'BTRA11', 'BTSG11', 'BTWR11', 'BZLI11', 'CACR11', 'CARE11', 'CBOP11', 'CCRF11', 'CEOC11', 'CJCT11', 'CNES11', 'CORM11', 'CORM11', 'CORM11', 'CORM11', 'CORM11', 'CORM11', 'CORM11', 'CORM11', 'CORM11', 'CPFF11', 'CPTR11', 'CPTS11', 'CRFF11', 'CTXT11', 'CVBI11', 'CXAG11', 'CXCE11B', 'CXCI11', 'CXCO11', 'CXRI11', 'CXTL11', 'CYCR11', 'DCRA11', 'DEVA11', 'DRIT11B', 'DVFF11', 'EDFO11B', 'EDGA11', 'EQIR11', 'ERCR11', 'ERCR11', 'ERCR11', 'ERCR11', 'ERCR11', 'ERCR11', 'ERCR11', 'ERCR11', 'ERCR11', 'ERCR11', 'ERCR11', 'ERCR11', 'ERCR11', 'ERCR11', 'ERCR11', 'ERCR11', 'ERPA11', 'EURO11', 'EVB

In [3]:
#@markdown Backup e listando colunas para limpeza. 

#   Backup
fii_df_bkp = fii_df

#@title Início da limpeza
fii_df.iloc[0]

Códigodo fundo                       FIVN11
Setor                             Shoppings
Preço Atual                         R$ 3,16
Liquidez Diária                      5205.0
Dividendo                           R$ 0,00
DividendYield                         0,00%
DY (3M)Acumulado                      0,00%
DY (6M)Acumulado                      0,00%
DY (12M)Acumulado                     0,00%
DY (3M)Média                          0,00%
DY (6M)Média                          0,00%
DY (12M)Média                         0,00%
DY Ano                                  NaN
Variação Preço                        0,00%
Rentab.Período                        0,00%
Rentab.Acumulada                      0,00%
PatrimônioLíq.             R$ 65.352.435,22
VPA                                 R$ 6,94
P/VPA                                  46.0
DYPatrimonial                           NaN
VariaçãoPatrimonial                     NaN
Rentab. Patr.no Período                 NaN
Rentab. Patr.Acumulada          

In [4]:
#@title Selecionando colunas
#@markdown Excluindo colunas consideradas irrelevantes.
new_fii_df = fii_df.drop(fii_df.columns[[1,2,3,5,6,7,8,9,10,11,12,13,14,15,18,19,20,21,22,23,24,25]], axis=1)
new_fii_df.info()
print('--------------LIMPEZA--------------')
new_fii_df.iloc[0]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 384 entries, 0 to 383
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Códigodo fundo  384 non-null    object
 1   Dividendo       384 non-null    object
 2   PatrimônioLíq.  375 non-null    object
 3   VPA             375 non-null    object
dtypes: object(4)
memory usage: 12.1+ KB
--------------LIMPEZA--------------


Códigodo fundo              FIVN11
Dividendo                  R$ 0,00
PatrimônioLíq.    R$ 65.352.435,22
VPA                        R$ 6,94
Name: 0, dtype: object

In [5]:
#@title Ajustando formatação
#@markdown Substituindo caracteres e formatação para compatibilidade.
#   R$ = [r raw] [^começa com] [\$ \string] [ <espaço>]
new_fii_df = new_fii_df.replace(to_replace=r'^R\$ ', value='',regex=True)
#   Separação de milhar (mantive vírgula para decimal)
new_fii_df['PatrimônioLíq.'] = new_fii_df['PatrimônioLíq.'].replace('.','')
#   Ignorado o ".0" na liquidez e "%" em demais colunas (abaixo exemplo que 'termina com %')
# new_fii_df = new_fii_df.replace(to_replace=r'%$ ', value='',regex=True)
new_fii_df.info()
new_fii_df.iloc[0]
# *não preciso transformar em float

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 384 entries, 0 to 383
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Códigodo fundo  384 non-null    object
 1   Dividendo       384 non-null    object
 2   PatrimônioLíq.  375 non-null    object
 3   VPA             375 non-null    object
dtypes: object(4)
memory usage: 12.1+ KB


Códigodo fundo           FIVN11
Dividendo                  0,00
PatrimônioLíq.    65.352.435,22
VPA                        6,94
Name: 0, dtype: object

In [6]:
#@title Concluindo formatação
#@markdown Renomeando colunas.
#   Renomeando Coluna
new_fii_df.rename(columns={'Códigodo fundo':'FII', 'Dividendo': 'Ult_Dv', 'PatrimônioLíq.': 'Pat_Lq','VPA':'VPA_Rnk'}, inplace=True)
#   Checando NaN
new_fii_df.isna().sum()

FII        0
Ult_Dv     0
Pat_Lq     9
VPA_Rnk    9
dtype: int64

In [7]:
#@title Listando Fii's (aleatórios do iFIX)
#@markdown Criando lista de Ativos de interesse.
fii_list = ['KNIP11','KNCR11','HGLG11','IRDM11','KNRI11','CPTS11','XPLG11','HCTR11','RECR11','MXRF11','HGRU11','BCFF11','BRCR11','HFOF11']

In [8]:
#@title Selecionando Ativos de interesse
#@markdown Dos danos do Web Scraping selecionando apenas ativos de interesse.
fiinal_df = new_fii_df[new_fii_df['FII'].isin(fii_list)]
fiinal_df

,FII,Ult_Dv,Pat_Lq,VPA_Rnk
112,BRCR11,"0,50","2.673.022.814,44","100,35"
117,RECR11,"0,74","2.520.731.349,40","95,33"
118,HCTR11,"1,10","2.681.663.950,61","121,43"
124,BCFF11,"0,56","1.903.260.829,67","75,57"
142,IRDM11,"0,70","3.389.714.938,91","93,04"
149,CPTS11,"0,37","2.858.127.870,96","89,93"
205,KNRI11,"1,00","3.867.931.624,08","160,12"
225,KNIP11,"0,94","7.645.646.524,33","94,80"
305,HGLG11,"2,20","3.466.816.466,82","147,80"
307,MXRF11,"0,10","2.282.646.736,06","10,10"


In [9]:
#@title Segundo Web Scraping.
#@markdown Dos ativos de interesse individualmente recolhendo informações acerca dos últimos dividendos.

# Linhas inativas são contadores internos para checar erros no processo (debug).

#import time
#from IPython.display import clear_output

dvs_fii = pd.DataFrame()
#x_len = len(fii_list)
#y_len = 0

for i in fii_list:
  #y_temp = y_len+1
  #y_len = y_temp
  zero = requests.get(f'https://www.fundsexplorer.com.br/funds/{i}')
  um = BeautifulSoup(zero.text,'html.parser')
  dois = um.findAll(attrs={'class':'table-responsive'})
  tres = dois[0].findAll('table')
  quatro = pd.read_html(str(tres))[0]
  cinco = quatro.drop(quatro.columns[5], axis=1)
  seis = cinco.drop(1)
  seis.loc[0,'Proventos'] = f'{i}'
  dvs_fii = dvs_fii.append(seis)
  #time.sleep(1)
  #clear_output(wait=True)
  #print(f'{y_temp}/{x_len}')
dvs_fii = dvs_fii.replace(to_replace=r'^R\$ ', value='',regex=True)
dvs_fii.rename(columns={'Proventos':'FII', 'Último': 'Ult_Dv_0', '3 meses': '3','6 meses':'6','12 meses':'12'}, inplace=True)
dvs_fii

,FII,Ult_Dv_0,3,6,12
0,KNIP11,"0,9400","1,7400","3,7100","12,4000"
0,KNCR11,"1,2000","3,4000","7,1000","13,1800"
0,HGLG11,"2,2000","4,4000","7,7000","16,5000"
0,IRDM11,"0,7016","2,3400","6,2191","13,7443"
0,KNRI11,"1,0000","2,8200","5,5500","10,5000"
0,CPTS11,"0,3700","2,2700","5,5700","12,1700"
0,XPLG11,"0,7400","2,2200","4,4000","8,4000"
0,HCTR11,"1,1000","3,3000","7,1500","16,6700"
0,RECR11,"0,7350","1,7773","4,7478","13,0406"
0,MXRF11,"0,1000","0,2800","0,6100","1,2100"


In [10]:
#@title Consolidando tabelas
#@markdown Mantido Ult_Dv_0 e Ult_Dv apenas para checar consistência interna de informações.
var01 = dvs_fii
var02 = fiinal_df
consolidado = pd.merge(var01,var02)
consolidado

,FII,Ult_Dv_0,3,6,12,Ult_Dv,Pat_Lq,VPA_Rnk
0,KNIP11,"0,9400","1,7400","3,7100","12,4000","0,94","7.645.646.524,33","94,80"
1,KNCR11,"1,2000","3,4000","7,1000","13,1800","1,20","5.747.307.324,41","100,66"
2,HGLG11,"2,2000","4,4000","7,7000","16,5000","2,20","3.466.816.466,82","147,80"
3,IRDM11,"0,7016","2,3400","6,2191","13,7443","0,70","3.389.714.938,91","93,04"
4,KNRI11,"1,0000","2,8200","5,5500","10,5000","1,00","3.867.931.624,08","160,12"
5,CPTS11,"0,3700","2,2700","5,5700","12,1700","0,37","2.858.127.870,96","89,93"
6,XPLG11,"0,7400","2,2200","4,4000","8,4000","0,74","3.102.791.995,75","114,49"
7,HCTR11,"1,1000","3,3000","7,1500","16,6700","1,10","2.681.663.950,61","121,43"
8,RECR11,"0,7350","1,7773","4,7478","13,0406","0,74","2.520.731.349,40","95,33"
9,MXRF11,"0,1000","0,2800","0,6100","1,2100","0,10","2.282.646.736,06","10,10"


In [11]:
#@title Exportando
#@markdown Exportando para utilização.

# Para utilizar excel:
# consolidado.to_excel(r'/fiiscraping.xlsx', sheet_name='Sheet1', index = False)

# Para notebook to html/github:
# !jupyter nbconvert --to html /*.ipynb